In [5]:
from pathlib import Path
from util import NUMBER_REGEX
import numpy as np

simon_dir = "sim/simon_simulations_20220721_torques/"
simon_path = Path(simon_dir)

sims = {}

c = 0
def p(*args, **kwargs):
    global c
    if c < 10:
        print(*args, **kwargs)
        c += 1

for sim_path in (i for j in zip(simon_path.glob("iopf_sim_DRAG*"), simon_path.glob("iopf_sim_torque*")) for i in j):
    label = sim_path.name
    slabel = label.split("_")
    drag_coefficient = int(slabel[slabel.index('DRAG')+1][0])
    n_particles = int(slabel[slabel.index('N')+1])
    seed = int(slabel[-1])
    torque = int("torque" in label)

    output_path = sim_path / "output.txt"
    with open(output_path, "r") as output_file:
        _t = 0
        _sysT = 0
        L = []
        for line in output_file:
            if line.startswith("t = "):
                t, N, sysT, *_ = map(float, NUMBER_REGEX.findall(line))

                if sysT > _sysT:
                    dt = t - _t
                    dsysT = sysT - _sysT
                    L.append((N, dt/dsysT))

                _t = t
                _sysT = sysT
    
    sims[label] = {
        "label": label,
        "drag_coefficient": drag_coefficient,
        "n_particles": n_particles,
        "seed": seed,
        "path": sim_path,
        "runtimes": np.array(L),
        "torque": torque
    }

    print(f"{label} {drag_coefficient=} {n_particles=} {seed=} {torque=}")

iopf_sim_DRAG_1.0_N_100_29381 drag_coefficient=1 n_particles=100 seed=29381 torque=0
iopf_sim_torque_DRAG_1.0_N_100_29381 drag_coefficient=1 n_particles=100 seed=29381 torque=1
iopf_sim_DRAG_1.0_N_100_47220 drag_coefficient=1 n_particles=100 seed=47220 torque=0
iopf_sim_torque_DRAG_1.0_N_100_47220 drag_coefficient=1 n_particles=100 seed=47220 torque=1
iopf_sim_DRAG_1.0_N_100_61392 drag_coefficient=1 n_particles=100 seed=61392 torque=0
iopf_sim_torque_DRAG_1.0_N_100_61392 drag_coefficient=1 n_particles=100 seed=61392 torque=1
iopf_sim_DRAG_1.0_N_100_92718 drag_coefficient=1 n_particles=100 seed=92718 torque=0
iopf_sim_torque_DRAG_1.0_N_100_92718 drag_coefficient=1 n_particles=100 seed=92718 torque=1


In [6]:
import rebound
from sim import data_io

def store_hdf5_rebound(buffer_rebound, sim):
    if sim.N < buffer_rebound.buf_x.shape[1]:
        buffer_rebound.flush()
        buffer_rebound.reset_buffer()
        buffer_rebound.initialize_buffer(sim.N)

    # Special routine to store rebound data into ABIE HDF5 format
    x = np.full(sim.N, np.nan)
    y = np.full(sim.N, np.nan)
    z = np.full(sim.N, np.nan)
    vx = np.full(sim.N, np.nan)
    vy = np.full(sim.N, np.nan)
    vz = np.full(sim.N, np.nan)
    masses = np.full(sim.N, np.nan)
    semi = np.full(sim.N, np.nan)
    ecc = np.full(sim.N, np.nan)
    inc = np.full(sim.N, np.nan)
    hashes = np.full(sim.N, -1, dtype=int)
    orbits = sim.calculate_orbits()
    for i in range(sim.N):
        x[i] = sim.particles[i].x
        y[i] = sim.particles[i].y
        z[i] = sim.particles[i].z
        vx[i] = sim.particles[i].vx
        vy[i] = sim.particles[i].vy
        vz[i] = sim.particles[i].vz
        masses[i] = sim.particles[i].m
        hashes[i] = sim.particles[i].hash.value
        if i > 0:
            semi[i] = orbits[i - 1].a
            ecc[i] = orbits[i - 1].e
            inc[i] = orbits[i - 1].inc
    pos = np.column_stack([x, y, z]).flatten()
    vel = np.column_stack([vx, vy, vz]).flatten()
    buffer_rebound.store_state(sim.t, pos=pos, vel=vel, masses=masses, a=semi, e=ecc, i=inc, names=hashes,
                                    energy=sim.calculate_energy())


In [9]:

for _, v in sims.items():
    seed = v["seed"]
    br_path = v["path"] / f"mercurius_data_reb_{seed}.h5"

    v["buffer_rebound"] = data_io.DataIO(output_file_name=str(br_path), CONST_G=4 * np.pi **2., append=False)

    sa = rebound.SimulationArchive(str(v["path"] / "mercurius_rebound_archive.bin"))

    v["buffer_rebound"].initialize_buffer(sa[0].N)

    for i in range(9800 if "bad" in v["label"] else 0, len(sa)):
        sim = sa[i]
        sim.G = 4 * np.pi ** 2
        store_hdf5_rebound(v["buffer_rebound"], sim)

    v["buffer_rebound"].close()
        

KeyboardInterrupt: 

In [10]:
for _, v in sims.items():
    seed = v["seed"]
    br_path = v["path"] / f"mercurius_data_reb_{seed}.h5"

    sa = rebound.SimulationArchive(str(v["path"] / "mercurius_rebound_archive.bin"))
    _t = float('inf')
    for i in range(len(sa)-1, -1, -1):
        s = sa[i]
        if s.t > _t:
            print(v["label"], i)
            break
        _t = s.t

In [8]:
from h5py import File
import gc

for obj in gc.get_objects():   # Browse through ALL objects
    if isinstance(obj, data_io.DataIO) or isinstance(obj, File):   # Just HDF5 files
        try:
            obj.close()
        except:
            pass # Was already closed